In [ ]:
import openpyxl
import os
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd 
from sklearn import metrics
import random
import gc 

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential

In [ ]:
# custom loss for the survival modeling using "nnet"
# refer to the following article for the nnet: A scalable discrete-time survival model for neural networks. PeerJ. 2019 Jan 25;7:e6257. doi: 10.7717/peerj.6257. eCollection 2019.
# for the following functions, "surv_likelihood" and "make_surv_array", copyrights belong to the original authors at https://github.com/MGensheimer/nnet-survival

def surv_likelihood(n_intervals):
    def loss(y_true, y_pred): 
        cens_uncens = 1. + y_true[:,0:n_intervals] * (y_pred-1.) #component for all individuals
        uncens = 1. - y_true[:,n_intervals:2*n_intervals] * y_pred #component for only uncensored individuals
        return keras.backend.sum(-keras.backend.log(keras.backend.clip(keras.backend.concatenate((cens_uncens,uncens)),keras.backend.epsilon(),None)),axis=-1) #return -log likelihood
    return loss
 
def make_surv_array(t,f,breaks): 
    n_samples=t.shape[0]
    #n_samples=len(t)
    n_intervals=len(breaks)-1
    timegap = breaks[1:] - breaks[:-1]
    breaks_midpoint = breaks[:-1] + 0.5*timegap
    y_train = np.zeros((n_samples,n_intervals*2))
    for i in range(n_samples):
        if f[i]: #if failed (not censored)
            y_train[i,0:n_intervals] = 1.0*(t[i]>=breaks[1:]) #give credit for surviving each time interval where failure time >= upper limit
            if t[i]<breaks[-1]: #if failure time is greater than end of last time interval, no time interval will have failure marked
                y_train[i,n_intervals+np.where(t[i]<breaks[1:])[0][0]]=1 #mark failure at first bin where survival time < upper break-point
        else: #if censored
            y_train[i,0:n_intervals] = 1.0*(t[i]>=breaks_midpoint) #if censored and lived more than half-way through interval, give credit for surviving the interval.
    return y_train 

In [ ]:
breaks = np.array([0, 12*3, 12*5, 12*10, 260])
n_intervals=len(breaks)-1

In [ ]:
os.chdir("/home/hk/Research/CXRage/")

In [ ]:
chosen="B5"    

In [ ]:
if chosen=="B0":
    imagesize=224
elif chosen=="B1":
    imagesize=240
elif chosen=="B2":
    imagesie=260
elif chosen=="B3":
    imagesize=300
elif chosen=="B4":
    imagesize=380
elif chosen=="B5":
    imagesize=456
elif chosen=="B6":
    imagesize=528
elif chosen=="B7":
    imagesize=600

In [ ]:
date=20230508

In [ ]:
paths=['./result/%d/FFT'%(date),
       './result/%d/FFT/bestmodel'%(date),
       './result/%d/FFT/finalmodel'%(date),
       './result/%d/bestmodel'%(date),
       './result/%d/finalmodel'%(date)       
      ]

In [ ]:
for path in paths:
    os.makedirs(path)

In [ ]:
def search(dirname): 
    data=[]
    try:
        filenames = os.listdir(dirname)
        for filename in filenames:
            full_filename = os.path.join(dirname, filename)
            if os.path.isdir(full_filename):
                search(full_filename)
            else:
                data.append(full_filename) 
    except PermissionError:
        pass
    return data

In [ ]:
def labelmaker(a1):    
    OS=[]
    death=[]
    num= len(a1)
    for i in range(num): 
        OS_ = a1[i].split("_")[-2]
        death_ = a1[i].split("_")[-1]
        death1_ = death_.split(".")[-2]
        OS.append(OS_)
        death.append(death1_)
    return OS, death

In [ ]:
def load_image(filename, label, augment=False): 
    raw = tf.io.read_file(filename)
    image = tf.image.decode_png(raw, channels=3)
    
    image.set_shape([512, 512, 3])  
    image = tf.image.resize(image, [imagesize, imagesize]) 
    
    if augment:
        image = tf.image.random_brightness(image, max_delta=25)
        image = tf.image.random_contrast(image, 0.7, 1.3)
        image = tf.clip_by_value(image, 0, 255)           
    
    label = tf.convert_to_tensor(label, dtype=tf.float32)
    
    return image, label

In [ ]:
def loadfrompath(path, dataset):
    data = search(path)
    OS, death = labelmaker(data)
    
    order = np.arange(len(data))
    np.random.shuffle(order) 
    
    data=np.array(data)
    OS = np.array(OS, dtype='float32')
    death = np.array(death, dtype='float32')
    
    data = data[order]
    OS=OS[order]
    death=death[order]
    
    label= make_surv_array(OS, death, breaks)
    label = np.float32(label)
    
    ds = tf.data.Dataset.from_tensor_slices( (data, label) )
    ds = ds.map(lambda x,y : load_image(x,y), num_parallel_calls=tf.data.AUTOTUNE)  
    
    data = pd.DataFrame(data)
    data.to_csv("/home/hk/Research/CXRage/result/%d/%s.csv"%(date, dataset))
    
    return ds

In [ ]:
def loadfrompath_train(path, dataset):
    data = search(path)
    OS, death = labelmaker(data)
    
    order = np.arange(len(data))
    np.random.shuffle(order) 
    
    data=np.array(data)
    OS = np.array(OS, dtype='float32')
    death = np.array(death, dtype='float32')
    
    data = data[order]
    OS=OS[order]
    death=death[order]
    
    label= make_surv_array(OS, death, breaks)
    label = np.float32(label)
    
    ds = tf.data.Dataset.from_tensor_slices( (data, label) )
    ds = ds.map(lambda x,y : load_image(x,y, augment=True), num_parallel_calls=tf.data.AUTOTUNE)  
    
    data = pd.DataFrame(data)
    data.to_csv("/home/hk/Research/CXRage/result/%d/%s.csv"%(date, dataset))
    
    return ds

In [ ]:
train_ds = loadfrompath_train('/home/hk/Research/CXRage/dataset0104/train', 'train')

In [ ]:
val_ds = loadfrompath('/home/hk/Research/CXRage/dataset0104/val' ,'val')

In [ ]:
test_ds = loadfrompath('/home/hk/Research/CXRage/dataset0104/test' ,'test')

In [ ]:
img_augmentation = Sequential(
    [tf.keras.layers.experimental.preprocessing.RandomZoom(height_factor=(-0.1, 0.1), fill_mode="constant"),   
     tf.keras.layers.experimental.preprocessing.RandomFlip(mode="horizontal"),
     tf.keras.layers.experimental.preprocessing.RandomTranslation(height_factor=(-0.05, 0.05), width_factor=(-0.05, 0.05), fill_mode="constant"),
     tf.keras.layers.experimental.preprocessing.RandomRotation(factor=(-0.05, 0.05), fill_mode="constant")
    ],
    name="img_augmentation")

In [ ]:
batch_size = 40

def configure_for_performance(ds):
    if chosen=="B0":
        ds = ds.cache() 
    ds = ds.shuffle(buffer_size=1000) 
    ds = ds.batch(batch_size, drop_remainder=True)
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

train_ds = configure_for_performance(train_ds)
val_ds = configure_for_performance(val_ds)
test_ds = configure_for_performance(test_ds)

In [ ]:
from tensorflow.keras.applications import EfficientNetB5

In [ ]:
cos_decay_ann = tf.keras.experimental.CosineDecayRestarts(initial_learning_rate=0.01, first_decay_steps=30, t_mul=2, m_mul=0.95, alpha=0.01)

In [ ]:
os.chdir("/home/hk/Research/CXRage/result/%d/"%(date))

In [ ]:
def build_model():
    inputs = layers.Input(shape=(imagesize, imagesize, 3))
    x = img_augmentation(inputs)
    model = EfficientNetB5(include_top=False, input_tensor=x, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)
    
    droprate=0.3

    x = layers.Dropout(droprate, name="dropout1")(x)
    x = layers.Dense(256, kernel_initializer='he_normal', activation="relu")(x)   
    
    x = layers.Dropout(droprate, name="dropout2")(x)
    x = layers.Dense(128, kernel_initializer='he_normal', activation="relu")(x)  
    
    x = layers.Dropout(droprate, name="dropout3")(x)
    x = layers.Dense(64, kernel_initializer='he_normal', activation="relu")(x)  
    
    x = layers.Dropout(droprate, name="dropout4")(x)
    outputs = layers.Dense(n_intervals, kernel_initializer='zeros', bias_initializer='zeros', activation="sigmoid", name="survpred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    model.compile(optimizer=keras.optimizers.SGD(learning_rate=cos_decay_ann),
                 loss=surv_likelihood(n_intervals))
    return model

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=40, min_delta= 0.0001) 
csv_logger = keras.callbacks.CSVLogger('./Model log.csv', append=False, separator=';')
checkpointer = keras.callbacks.ModelCheckpoint(filepath='/home/hk/Research/CXRage/result/%d/bestmodel'%(date), 
                                               verbose=1, 
                                               save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()

In [ ]:
epochs = 1000
with mirrored_strategy.scope():
    model = build_model()     
hist = model.fit(train_ds, epochs=epochs, validation_data=val_ds, 
                 callbacks=[early_stopping, csv_logger, checkpointer],
                 verbose=1) 

In [ ]:
model.save('/home/hk/Research/CXRage/result/%d/finalmodel'%(date))
model.save('CXRage%d.h5'%(date))

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
################################# Full fine-tuning #################################

In [ ]:
def unfreeze_model(model):
    for layer in model.layers:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

In [ ]:
cos_decay_ann = tf.keras.experimental.CosineDecayRestarts(initial_learning_rate=0.001, first_decay_steps=40, t_mul=2, m_mul=0.95, alpha=0.01)

In [ ]:
os.chdir("/home/hk/Research/CXRage/result/%d/FFT"%(date))

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=40, min_delta= 0.0001) 
csv_logger = keras.callbacks.CSVLogger('./Model log.csv', append=False, separator=';')
checkpointer = keras.callbacks.ModelCheckpoint(filepath='/home/hk/Research/CXRage/result/%d/FFT/bestmodel'%(date), 
                                               verbose=1, 
                                               save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
epochs = 1000
with mirrored_strategy.scope(): 
    unfreeze_model(model)    
    model.compile(optimizer=keras.optimizers.SGD(learning_rate=cos_decay_ann),
                  loss=surv_likelihood(n_intervals))

In [ ]:
hist = model.fit(train_ds, epochs=epochs, validation_data=val_ds, 
                  callbacks=[early_stopping, csv_logger, checkpointer],
                  verbose=1) 

In [ ]:
model.save('/home/hk/Research/CXRage/result/%d/FFT/finalmodel'%(date))
model.save('CXRage%dFFT.h5'%(date))

In [ ]:
hist1 = pd.DataFrame(hist.history)
hist1['epoch'] = hist.epoch
hist1.to_csv('history.csv', index=False, header=True)